## hyps aggregation

In [3]:
import pandas as pd 
fivefolds_label = pd.read_csv('data/biclass2.txt', sep=' ')
fivefolds_label.rename(columns = {'file':'name'}, inplace = True)
fivefolds_label['name'] =  fivefolds_label['name'].str.replace('CTT', '').astype(int)
fivefolds_label['label'] =  fivefolds_label['label'].astype(bool).astype(str)
fivefolds_label['label'] =  fivefolds_label['label'].str.replace('False', 'NC')
fivefolds_label['label'] =  fivefolds_label['label'].str.replace('True', 'AD')

In [4]:
bert_v2_15k = pd.read_csv('biclass_result/bert-v2-15k-mean4.csv')
bert_v2_15k_hyps = [float(i) for i in bert_v2_15k['hyps'].tolist()]
bert_v2_15k_truth = [float(i) for i in bert_v2_15k['truth'].tolist()]
bert_v2_15k_name = [str(i).replace("CTT", "") for i in bert_v2_15k['name'].tolist()]
print(bert_v2_15k_hyps)

[0.7801008224487305, 0.9378973245620728, 0.9035855531692504, 0.2711054980754852, 0.8930366635322571, 0.9770812392234802, 0.0404171869158744, 0.1090722531080246, 0.6683647036552429, 0.2028627544641494, 0.8719687461853027, 0.0832266807556152, 0.0362274721264839, 0.0629442185163497, 0.1242963001132011, 0.0356787741184234, 0.2175018489360809, 0.0557883940637111, 0.2191504091024398, 0.3267900645732879, 0.8766983151435852, 0.0405016057193279, 0.3883920609951019, 0.3821354210376739, 0.280805379152298, 0.7967990636825562, 0.8061067461967468, 0.3468203246593475, 0.2447982132434845, 0.2100969105958938, 0.0312201995402574, 0.054668691009283, 0.114611268043518, 0.2750318646430969, 0.1947158277034759, 0.0123608987778425, 0.0187535751610994, 0.3540928959846496, 0.0733232349157333, 0.7018376588821411, 0.0287108328193426, 0.1649993658065796, 0.2485504001379013, 0.0373977422714233, 0.0750066712498664, 0.5406267046928406, 0.0456023551523685, 0.2784138023853302, 0.021545473486185, 0.0653745234012603, 0.1

In [5]:
whisper_v2_15k = pd.read_csv('biclass_result/whisper-WhisperForAudioClassification-v2-15k_2.csv')
whisper_v2_15k_hyps = [float(i) for i in whisper_v2_15k['hyps'].tolist()]
whisper_v2_15k_truth = [float(i) for i in whisper_v2_15k['truth'].tolist()]
whisper_v2_15k_name = [str(i).replace("CTT", "") for i in whisper_v2_15k['name'].tolist()]
print(whisper_v2_15k_truth)

[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [6]:
llama_v2 = pd.read_csv('biclass_result/llama_v2.csv')
llama_v2_hyps = [float(i) for i in llama_v2['hyps'].tolist()]
llama_v2_truth = [float(i) for i in llama_v2['truth'].tolist()]
llama_v2_name = [str(i).replace("CTT", "") for i in llama_v2['name'].tolist()]
print(llama_v2_hyps)

FileNotFoundError: [Errno 2] No such file or directory: 'biclass_result/llama_v2.csv'

## model

In [1]:
import torch
torch.cuda.is_available()
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
import torch
import argparse
import numpy as np
import pandas as pd

dfs = []
alldf =pd.DataFrame()

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-whisperclass-test.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',0)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', True)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 9)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
    force_cudnn_initialization()

    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            # from test_whisperclass import Solver
            from Whisper_Biclass_concat_test import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            assert paras.load is not None
            from train_binaryclass2_5folds2 import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))
    for idx in range(5):
        paras.config = f'./config/cv11Lu_asr_lstm4atthead_allvocab-biclass-test-5-{idx+1}.yaml'
        config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
        config['core']['ckpt'] = f'LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-{idx+1}_sd0/combine-v2-15k-a-o-d-final.pth'
        solver = Solver(config, paras, mode)

        solver.load_data()
    #     solver.print_model()
        solver.set_model()
        names, hyps, txts, ans = solver.exec()
        dfs.append(pd.DataFrame(
            {'name': names,
             'hyps': hyps,
             'truth': txts,
             'ifcorrect': ans
            }))
        print(dfs[idx])
        print(names)
        print(hyps)
        print(txts)
        print(ans)
        del solver


0


In [2]:
main()


Using test mode

[INFO] Evaluating result of tr. config @ ../LAS_Mandarin_PyTorch-master/config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1.yaml
[INFO] Evaluating result of tr. ckpt @ LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1_sd0/combine-v2-15k-a-o-d-final.pth


c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_r

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-1-14']
Mozillacv11Dataset CTT5-1-14 found wav data: 30
text len: 30
remove None, then wav data: 30, text len: 30
[VAGDataset] path: data_process, split: ['CTTsegment_diarization_overlap']
Mozillacv11Dataset CTTsegment_diarization_overlap found wav data: 166
text len: 166
remove None, then wav data: 165, text len: 165
Setup ASR model and optimizer 
('data_process\\CTT5-1-14\\15CTT_1.wav',)   0.643486738204956   0.0   False
('data_process\\CTT5-1-14\\15CTT_2.wav',)   0.6556360721588135   0.0   False
('data_process\\CTT5-1-14\\16CTT_1.wav',)   0.8335856795310974   1.0   True
('data_process\\CTT5-1-14\\22CTT_1.wav',)   0.26010555028915405   0.0   True
('data_process\\CTT5-1-14\\28CTT_1.wav',)   0.6020953059196472   1.0   True
('data_process\\CTT5-1-14\\34CT

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a Be

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-2-14']
Mozillacv11Dataset CTT5-2-14 found wav data: 26
text len: 26
remove None, then wav data: 26, text len: 26
[VAGDataset] path: data_process, split: ['CTTsegment_diarization_overlap']
Mozillacv11Dataset CTTsegment_diarization_overlap found wav data: 166
text len: 166
remove None, then wav data: 165, text len: 165
Setup ASR model and optimizer 
('data_process\\CTT5-2-14\\11CTT_1.wav',)   0.7810555100440979   1.0   True
('data_process\\CTT5-2-14\\25CTT_1.wav',)   0.5507970452308655   1.0   True
('data_process\\CTT5-2-14\\2CTT_1.wav',)   0.23674210906028748   0.0   True
('data_process\\CTT5-2-14\\2CTT_2.wav',)   0.40972787141799927   0.0   True
('data_process\\CTT5-2-14\\37CTT_1.wav',)   0.4506596624851227   1.0   False
('data_process\\CTT5-2-14\\39CTT

...

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-3-14']
Mozillacv11Dataset CTT5-3-14 found wav data: 30
text len: 30
remove None, then wav data: 30, text len: 30
[VAGDataset] path: data_process, split: ['CTTsegment_diarization_overlap']
Mozillacv11Dataset CTTsegment_diarization_overlap found wav data: 166
text len: 166
remove None, then wav data: 165, text len: 165
Setup ASR model and optimizer 
('data_process\\CTT5-3-14\\18CTT_1.wav',)   0.7429232597351074   1.0   True
('data_process\\CTT5-3-14\\18CTT_2.wav',)   0.8441930413246155   1.0   True
('data_process\\CTT5-3-14\\1CTT_1.wav',)   0.22037553787231445   0.0   True
('data_process\\CTT5-3-14\\1CTT_2.wav',)   0.369803249835968   0.0   True
('data_process\\CTT5-3-14\\20CTT_1.wav',)   0.6164882183074951   1.0   True
('data_process\\CTT5-3-14\\30CTT_1.

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a Be

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-4-14']
Mozillacv11Dataset CTT5-4-14 found wav data: 30
text len: 30
remove None, then wav data: 30, text len: 30
[VAGDataset] path: data_process, split: ['CTTsegment_diarization_overlap']
Mozillacv11Dataset CTTsegment_diarization_overlap found wav data: 166
text len: 166
remove None, then wav data: 165, text len: 165
Setup ASR model and optimizer 
('data_process\\CTT5-4-14\\100CTT_1.wav',)   0.4535585343837738   0.0   True
('data_process\\CTT5-4-14\\100CTT_2.wav',)   0.4571946859359741   0.0   True
('data_process\\CTT5-4-14\\12CTT_1.wav',)   0.6792091727256775   1.0   True
('data_process\\CTT5-4-14\\17CTT_1.wav',)   0.589993417263031   1.0   True
('data_process\\CTT5-4-14\\21CTT_1.wav',)   0.540230393409729   0.0   False
('data_process\\CTT5-4-14\\29CTT

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a Be

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: data_process, split: ['CTT5-5-14']
Mozillacv11Dataset CTT5-5-14 found wav data: 28
text len: 28
remove None, then wav data: 28, text len: 28
[VAGDataset] path: data_process, split: ['CTTsegment_diarization_overlap']
Mozillacv11Dataset CTTsegment_diarization_overlap found wav data: 166
text len: 166
remove None, then wav data: 165, text len: 165
Setup ASR model and optimizer 
('data_process\\CTT5-5-14\\10CTT_1.wav',)   0.4939834773540497   1.0   False
('data_process\\CTT5-5-14\\13CTT_1.wav',)   0.61575847864151   0.0   False
('data_process\\CTT5-5-14\\13CTT_2.wav',)   0.6691688299179077   0.0   False
('data_process\\CTT5-5-14\\13CTT_3.wav',)   0.6218217611312866   0.0   False
('data_process\\CTT5-5-14\\13CTT_4.wav',)   0.6061222553253174   0.0   False
('data_process\\CTT5-5-14\\13C

In [3]:
for idx, df in enumerate(dfs):
    alldf = pd.concat([alldf, df])
    # df.to_csv(f'./biclass_result/whisper-WhisperForAudioClassification-v3-10k-{idx+1}_2', index=False)
alldf.to_csv(f'./biclass_result/combine-v2-15k-a-o-d-final.csv', index=False)
# whisper-v2-10k-{idx+1}_with_id_2.csv
# whisper-v2-15k_with_id_2.csv
# whisper-WhisperForAudioClassification-v3-10k-{idx+1}_2
# whisper-WhisperForAudioClassification-v3-10k_2.csv